# Hand Gesture Classification
### by Adrian Abraham

## 1 - Package installation
- **tensorflow**: for model creation and predictions
- **pandas**: for csv_reading
- **numpy**: for data storage and manipulation
- **mediapipe**: contains pre-made hand detection module for data collection
- **open-cv**: for live feed
- **ast**: to convert string literal lists into actual lists

In [15]:
!pip install tensorflow

In [16]:
!pip install pandas

In [17]:
!pip install mediapipe

In [18]:
!pip install opencv-python

## 2 - Importing packages

In [19]:
import tensorflow as tf
import numpy as np
import pandas as pd
import ast
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.losses import SparseCategoricalCrossentropy

### Quick check that tensorflow is working

In [20]:
print(tf.__version__)

2.16.2


## 3 - Importing data

In [21]:
# Positive and negative data are stored in csv format.
# We can use read_csv() to get the contents of each file
peace_set = pd.read_csv('peace.csv')
heart_set = pd.read_csv('heart.csv')
shaka_set = pd.read_csv('shaka.csv')
none_set  = pd.read_csv('none.csv' )

In [22]:
peace_set.info(), heart_set.info(), shaka_set.info(), none_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1984 entries, 0 to 1983
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   coords  1984 non-null   object
dtypes: object(1)
memory usage: 15.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1752 entries, 0 to 1751
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   coords  1752 non-null   object
dtypes: object(1)
memory usage: 13.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1513 entries, 0 to 1512
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   coords  1513 non-null   object
dtypes: object(1)
memory usage: 11.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2245 entries, 0 to 2244
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   coords  2245 non-null   object
dtypes: object(1)
mem

(None, None, None, None)

In [23]:
# The format of our csv has a list of data as a string, 
# to turn them into lists so we use the ast library
peace_set['coords'] = peace_set['coords'].apply(ast.literal_eval)
peace_set = np.array(peace_set['coords'].tolist())

heart_set['coords'] = heart_set['coords'].apply(ast.literal_eval)
heart_set = np.array(heart_set['coords'].tolist())

shaka_set['coords'] = shaka_set['coords'].apply(ast.literal_eval)
shaka_set = np.array(shaka_set['coords'].tolist())

none_set['coords'] = none_set['coords'].apply(ast.literal_eval)
none_set = np.array(none_set['coords'].tolist())

In [24]:
# We can concatenate them together to have all our data in one np array
data = np.concatenate((peace_set,heart_set,shaka_set,none_set))
data.shape

(7494, 21, 2)

In [25]:
gestures = ["peace","heart","shaka","none"]

In [26]:
target1 = np.zeros(peace_set.shape[0])    # Class 0 for data1
target2 = np.ones(heart_set.shape[0])     # Class 1 for data2
target3 = np.full(shaka_set.shape[0], 2)  # Class 2 for data3
target4 = np.full(none_set.shape[0], 3)

targets = np.concatenate((target1, target2, target3, target4))
targets

array([0., 0., 0., ..., 3., 3., 3.])

In [27]:
model = Sequential([
    Input(shape=data.shape[1:]),              # Define the input shape here
    Flatten(),                                # Flatten the (21, 2) input into a 1D vector
    Dense(32, activation='relu',  name='L1'), # Fully connected layer with 25 neurons
    Dense(16, activation='relu',  name='L2'), # Fully connected layer with 16 neurons
    Dense(4, activation='linear', name='L3')  # Output layer for binary classification
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L1 (Dense)                      │ (None, 32)             │         1,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L2 (Dense)                      │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L3 (Dense)                      │ (None, 4)              │            68 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,972 (7.70 KB)

 Trainable params: 1,972 (7.70 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=SparseCategoricalCrossentropy(from_logits=True),  # Use appropriate loss function for classification
              metrics=['accuracy'])

In [29]:
history = model.fit(
    data,targets,
    epochs=40
)

Epoch 1/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 450us/step - accuracy: 0.6875 - loss: 1.0600
Epoch 2/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 429us/step - accuracy: 0.9056 - loss: 0.3007
Epoch 3/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 425us/step - accuracy: 0.9393 - loss: 0.1903
Epoch 4/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step - accuracy: 0.9583 - loss: 0.1305
Epoch 5/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - accuracy: 0.9648 - loss: 0.1005
Epoch 6/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step - accuracy: 0.9761 - loss: 0.0727
Epoch 7/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 434us/step - accuracy: 0.9827 - loss: 0.0622
Epoch 8/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step - accuracy: 0.9865 - loss: 0.0506
Epoch 9/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - accuracy: 0.9891 - loss: 0.0438
Epoch 10/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step - accuracy: 0.9914 - loss: 0.0328
Epoch 11/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step - accuracy: 0.9929 - loss: 0.0326
Epoch 12/40
235/235 ━━━━━━━━━━

In [30]:
model.save("gesture_model.keras")